### IF4091 Pembelajaran Mesin
## Tugas Besar II: Feed Forward Neural Network
**Nama : Erick Wijaya**

**NIM: 13515057**

**Kelas: K01**

# Import Statement

In [108]:
from copy import deepcopy
from keras import optimizers
from keras.layers import Dense, Activation
from keras.models import Sequential
# sklearn for evaluation
from sklearn.model_selection import KFold, train_test_split

import numpy as np
import pandas as pd

# Preprocessing Functions

In [109]:
def onehot_encode(df, colname):
    uniques = np.unique(df[colname]).tolist()
    dict_cols = {key: np.zeros((df.shape[0],), dtype=int) for key in uniques}
    for i, row in df.iterrows():
        for key in dict_cols:
            if row[colname] == key:
                dict_cols[key][i] = 1
                break
    for key, val in dict_cols.items():
        df["_" + key] = val
    return df, list(dict_cols.keys())

def label_encode(df, colname):
    uniques = np.unique(df[colname]).tolist()
    dict_unique = {key: i for i, key in enumerate(uniques)}
    rows = [dict_unique[row[colname]] for _, row in df.iterrows()]
    df["_" + colname] = np.array(rows)
    return df, dict_unique

def zscore_encode(df, colname):
    mean = np.mean(df[colname])
    std = np.std(df[colname])
    df["z_" + colname] = np.array([(row[colname]-mean)/std for _, row in df.iterrows()])
    return df, mean, std

# Preprocess

In [139]:
# Load Data
df = pd.read_csv("weather.txt")
print("Raw: (head)"); display(df.head(2))

# Preprocess
df, values_outlook = onehot_encode(df, "outlook")
df, dict_windy = label_encode(df, "windy")
df, dict_play = label_encode(df, "play")
df, mean_temp, std_temp = zscore_encode(df, "temperature")
df, mean_hum, std_hum = zscore_encode(df, "humidity")
labels = df["_play"].values
del df["outlook"], df["windy"], df["play"], df["temperature"], df["humidity"]

# Split Data 80/10/10
train_df, validation_df, test_df = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])
train_labels = train_df["_play"].values
validation_labels = validation_df["_play"].values
test_labels = test_df["_play"].values
del train_df["_play"], validation_df["_play"], test_df["_play"]
print("Train:"); display(train_df)
print("Validation:"); display(validation_df)
print("Test:"); display(test_df)

print(values_outlook, dict_windy, dict_play)
print("Labels:", labels)

Raw: (head)


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no


Train:


,_overcast,_rainy,_sunny,_windy,z_temperature,z_humidity
0,0,0,1,0,1.804715,0.338726
2,1,0,0,0,1.488890,0.439623
6,1,0,0,1,-1.511449,-1.679217
3,0,1,0,0,-0.563974,1.448595
8,0,0,1,0,-0.721886,-1.174731
10,0,0,1,1,0.225589,-1.174731
11,1,0,0,1,-0.248148,0.843212
1,0,0,1,1,1.015152,0.843212
4,0,1,0,0,-0.879799,-0.165760
13,0,1,0,1,-0.406061,0.944109


Validation:


,_overcast,_rainy,_sunny,_windy,z_temperature,z_humidity
7,0,0,1,0,-0.248148,1.347697


Test:


,_overcast,_rainy,_sunny,_windy,z_temperature,z_humidity
9,0,1,0,0,0.225589,-0.165760
12,1,0,0,0,1.173065,-0.670245


['overcast', 'rainy', 'sunny'] {False: 0, True: 1} {'no': 0, 'yes': 1}
Labels: [0 0 1 1 1 0 1 0 1 1 1 1 1 0]


# Classifiers

## Backpropagation

In [72]:
class NNClassifier:
    def __init__(self, n_nodes=[], lrate=0.05, momentum=0, batch_size=1, max_iter=100):
        print(self.__MAX_HIDDEN)
        # Checking parameter input
        if (len(n_nodes) > self.__MAX_HIDDEN):
            raise ValueError('Number of hidden layers cannot be greater than {}'.format(self.__MAX_HIDDEN))

        if (not all(x > 0 for x in n_nodes)):
            raise ValueError('Number of nodes in a layer cannot be nonpositive')

        if (batch_size <= 0):
            raise ValueError('Batch size cannot be nonpositive')

        # Setting parameter
        self.n_nodes = n_nodes
        self.n_hiddens = len(n_nodes)
        self.lrate = lrate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.weights = []
        self.prev_weights = []

    @property
    def __MAX_HIDDEN(self):
        return 10

    def __stochastic_gradient_descend(self, data, target):
        for x, y in zip(data, target):
            values_layers = self.__feed_forward(x)
            errors_layers = self.__backward_prop(y, values_layers)
            values_layers.insert(0, x)

            # Update weight
            new_weights = []
            for ilayer, (weights_per_layer, prev_weights_per_layer) in enumerate(zip(self.weights, self.prev_weights)):
                new_weights_per_layer = []
                for inode, (weight_all, prev_weight_all) in enumerate(zip(weights_per_layer, prev_weights_per_layer)):
                    new_weight_all = []
                    for iweight, (weight, prev_weight) in enumerate(zip(weight_all, prev_weight_all)):
                        new_weight_all.append(self.__calculate_weight(weight, prev_weight, 
                        values_layers[ilayer][inode], errors_layers[ilayer][iweight]))
                    new_weights_per_layer.append(new_weight_all)
                new_weights.append(np.array(new_weights_per_layer))
            self.prev_weights = deepcopy(self.weights)
            self.weights = new_weights
    
    def __feed_forward(self, x):
        outputs = [x]
        for weight in self.weights:
            outputs.append(self.__sigmoid(outputs[-1] @ weight))
        del outputs[0]
        return outputs
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def __backward_prop(self, target, values_layers):
        n_hiddens_out_layers = len(values_layers)
        errors_layers = [None] * n_hiddens_out_layers
        for i in range(n_hiddens_out_layers-1, 0-1, -1):
            errors = []
            if i < n_hiddens_out_layers-1: # (hidden layer)
                for inode, output in enumerate(values_layers[i]):
                    errors.append(self.__hidden_error(output, inode, i, errors_layers))
            else: # i == n_hiddens_out_layers-1 (output layer)
                for output in values_layers[i]:
                    errors.append(self.__output_error(output, target))
            errors_layers[i] = np.array(errors)
        return errors_layers

    def __output_error(self, output, target):
        return output * (1 - output) * (target - output)
    
    def __hidden_error(self, output, inode, index_layer, errors_layers):
        index_delta = index_layer + 1
        index_weight = index_layer + 1
        sigma = 0
        for i in range(0, len(self.weights[index_weight][inode])):
            # takut salah indexnya
            sigma += self.weights[index_weight][inode][i] * errors_layers[index_delta][i]
        return output * (1 - output) * sigma

    def __calculate_weight(self, weight, prev_weight, err, val):
        return weight + self.momentum * prev_weight + self.lrate * err * val

    def fit(self, data, target):
        self.__initialize_weights(data)
        print(self.weights)

        for _ in range(self.max_iter):
            # Random shuffle data and target simultaneously
            p = np.random.permutation(data.shape[0])
            data, target = data[p], target[p]

            # Do gradient descent per batch
            for i in range(0, data.shape[0], self.batch_size):
                index = list(range(i, i+self.batch_size))
                self.__stochastic_gradient_descend(data[index], target[index])
            
        print(self.weights)
        return self

    def __initialize_weights(self, data):
        # Initialize weights with random numbers
        n_features = data.shape[1]
        if (self.n_hiddens > 0):
            self.weights = [np.random.randn(n_features, self.n_nodes[0])]
            for i in range(1, self.n_hiddens):
                self.weights.append(np.random.randn(self.n_nodes[i-1], self.n_nodes[i]))
            self.weights.append(np.random.randn(self.n_nodes[self.n_hiddens - 1], 1))
        else:
            self.weights = [np.random.randn(n_features, 1)]
        
        # Assume first prev_weights be zeroes
        self.prev_weights = deepcopy(self.weights)
        for i, prev_weight_per_layer in enumerate(self.prev_weights):
            for j, prev_weight_all in enumerate(prev_weight_per_layer):
                for k, _ in enumerate(prev_weight_all):
                    self.prev_weights[i][j][k] = 0

## Keras

In [73]:
class NNKeras():
    def __init__(self, nnodes_per_hidden_layer=[100], lrate=0.05, momentum=0, batch_size=1):
        self.nnodes_per_hidden_layer = nnodes_per_hidden_layer
        self.lrate = lrate
        self.momentum = momentum
        self.batch_size = batch_size
    
    def fit(self, data, labels, epochs=1):
        """data: ndarray"""
        n_rows = len(data)
        n_attr = len(data[n_rows-1])
        self.model = Sequential()
        # First Hidden Layer
        self.model.add(Dense(units=self.nnodes_per_hidden_layer[0], activation='sigmoid', input_dim=n_attr))
        # 2nd .. Last Hidden Layer
        for i in range(1, len(self.nnodes_per_hidden_layer)):
            self.model.add(Dense(units=self.nnodes_per_hidden_layer[i], activation='sigmoid'))
        # Output Layer
        self.model.add(Dense(units=1, activation='sigmoid'))
        
        sgd = optimizers.SGD(lr=self.lrate, momentum=self.momentum)
        self.model.compile(optimizer=sgd, loss='mean_squared_error')
        self.model.fit(data, labels, batch_size=self.batch_size, epochs=epochs)
        return self

    def evaluate(self, test_data, test_labels):
        return self.model.evaluate(test_data, test_labels, batch_size=self.batch_size)

    def predict(self, sample):
        return self.model.predict(sample, batch_size=self.batch_size)

# Training (Hold-Out)

## Backpropagation

## Keras

In [115]:
model_keras = NNKeras(nnodes_per_hidden_layer=[64, 32, 16, 16], lrate=0.05, momentum=0.01, batch_size=14).fit(
    train_df.values, labels, epochs=150)

Epoch 1/150
14/14 [==============================] - 0s 33ms/step - loss: 0.3699
Epoch 2/150
14/14 [==============================] - 0s 125us/step - loss: 0.3620
Epoch 3/150
14/14 [==============================] - 0s 140us/step - loss: 0.3542
Epoch 4/150
14/14 [==============================] - 0s 257us/step - loss: 0.3464
Epoch 5/150
14/14 [==============================] - 0s 1ms/step - loss: 0.3389
Epoch 6/150
14/14 [==============================] - 0s 187us/step - loss: 0.3316
Epoch 7/150
14/14 [==============================] - 0s 213us/step - loss: 0.3245
Epoch 8/150
14/14 [==============================] - 0s 137us/step - loss: 0.3177
Epoch 9/150
14/14 [==============================] - 0s 146us/step - loss: 0.3111
Epoch 10/150
14/14 [==============================] - 0s 341us/step - loss: 0.3049
Epoch 11/150
14/14 [==============================] - 0s 144us/step - loss: 0.2991
Epoch 12/150
14/14 [==============================] - 0s 123us/step - loss: 0.2935
Epoch 13/150
14/

Epoch 100/150
14/14 [==============================] - 0s 148us/step - loss: 0.2293
Epoch 101/150
14/14 [==============================] - 0s 139us/step - loss: 0.2293
Epoch 102/150
14/14 [==============================] - 0s 126us/step - loss: 0.2293
Epoch 103/150
14/14 [==============================] - 0s 117us/step - loss: 0.2293
Epoch 104/150
14/14 [==============================] - 0s 119us/step - loss: 0.2293
Epoch 105/150
14/14 [==============================] - 0s 128us/step - loss: 0.2293
Epoch 106/150
14/14 [==============================] - 0s 118us/step - loss: 0.2293
Epoch 107/150
14/14 [==============================] - 0s 118us/step - loss: 0.2293
Epoch 108/150
14/14 [==============================] - 0s 118us/step - loss: 0.2293
Epoch 109/150
14/14 [==============================] - 0s 117us/step - loss: 0.2293
Epoch 110/150
14/14 [==============================] - 0s 112us/step - loss: 0.2293
Epoch 111/150
14/14 [==============================] - 0s 298us/step - loss: